In [1]:
import torch
import syft as sy
import pandas as pd
from syft.lib.python.list import List
print(sy.__version__)
print(torch.__version__)

0.5.0
1.8.1+cu102


### Make the VM, its clients, and send some data to each of them

In [2]:
server = sy.VirtualMachine(name="server")
root_client = server.get_root_client()
client = server.get_client()

x = torch.tensor([1,2,3]).tag("hey", "there", "buddy")
x.description = "wow what a day"
x2 = torch.tensor([4,5,6]).tag("hello", "my", "friend")
x2.description = "yes great weather"

x_ptr = x.send(root_client)
x2_ptr = x2.send(client)

The tensors are visible to both clients since its in the store.

In [3]:
root_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6303007b1b194354a6a9a02f9b103f94>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>
1,<UID: e0037c68c1e24c5e8a3b4ffc9283d963>,"[hello, my, friend]",yes great weather,<class 'torch.Tensor'>


In [4]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6303007b1b194354a6a9a02f9b103f94>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>
1,<UID: e0037c68c1e24c5e8a3b4ffc9283d963>,"[hello, my, friend]",yes great weather,<class 'torch.Tensor'>


What if the normal client try to get the data sent to the root client?

In [5]:
try:
    ptr1 = client.store[0]
    ptr1.get_copy()
except Exception as e:
    print(e)

[2021-09-16T08:17:16.353965+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: 6303007b1b194354a6a9a02f9b103f94>Please submit a request.
[2021-09-16T08:17:16.354664+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: 6303007b1b194354a6a9a02f9b103f94>Please submit a request.
[2021-09-16T08:17:16.356068+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: 6303007b1b194354a6a9a02f9b103f94>Please submit a request.


You do not have permission to .get() Object with ID: <UID: 6303007b1b194354a6a9a02f9b103f94>Please submit a request.


Root client can get the first data in the store

In [6]:
ptr2 = root_client.store[0]
ptr2.get_copy()

tensor([1, 2, 3])

The root client, however, can get the data sent to the normal client

In [7]:
try:
    ptr3 = root_client.store[1]
    print(ptr3.get_copy())
except Exception as e:
    print(e)

tensor([4, 5, 6])


The normal client can also get its data

In [8]:
ptr4 = client.store[1]
ptr4.get_copy()

tensor([4, 5, 6])

Explanation rom [Madhava Jay](http://github.com/madhavajay): when you send data in, it has your key attached to it for `read` permission. 
If you try to read data which doesn't have your key attached it wont work 
but if you request and a `root` user approves the request then your key gets 
added to the data item. When ever you do execution its generally allowed by 
anyone and the result is stored in a new data object. That new data object 
has the intersection of the keys that were on each item used in the computation 
so for example `a + b` will produce `c` with the intersection of the keys on `a` and 
keys on `b`. The result is that only people who had permission to read `a` and `b` 
already will be able to read `c`, so the moment you mix data with which you do 
not have read permission you will no longer be able to get the result freely 
without making a request.

In [9]:
print(type(server))
print(server)
print(type(root_client))
print(root_client)
print(client)

<class 'syft.core.node.vm.vm.VirtualMachine'>
VirtualMachine: server: <UID: 628979738efb4fa899fbea5fc9f03364>
<class 'syft.core.node.vm.client.VirtualMachineClient'>
<VirtualMachineClient: server Client>
<VirtualMachineClient: server Client>


### Check out the remote modules

In [10]:
remote_pd = root_client.pandas
remote_torch = root_client.torch
remote_np = root_client.numpy
remote_vision = root_client.torchvision

In [11]:
print(type(remote_pd))
remote_pd

<class 'syft.ast.module.Module'>


Module:
	.DataFrame -> <syft.ast.klass.Class object at 0x7f85a26cd640>
	.Series -> <syft.ast.klass.Class object at 0x7f85a26cd6a0>
	.CategoricalDtype -> <syft.ast.klass.Class object at 0x7f85a26cd700>
	.Categorical -> <syft.ast.klass.Class object at 0x7f85a26cd8e0>
	.json_normalize -> <syft.ast.callable.Callable object at 0x7f85a26cd9a0>

In [12]:
remote_np

Module:
	.ndarray -> <syft.ast.klass.Class object at 0x7f85a25c5700>

In [13]:
remote_vision

Module:
	.__version__ -> <syft.ast.static_attr.StaticAttribute object at 0x7f85a27f4400>
	.transforms -> Module:
		.Compose -> <syft.ast.klass.Class object at 0x7f85a27f44c0>
		.ToTensor -> <syft.ast.klass.Class object at 0x7f85a27f4520>
		.Normalize -> <syft.ast.klass.Class object at 0x7f85a27f45e0>
		.CenterCrop -> <syft.ast.klass.Class object at 0x7f85a26a89a0>
		.ColorJitter -> <syft.ast.klass.Class object at 0x7f85a26a8a60>
		.FiveCrop -> <syft.ast.klass.Class object at 0x7f85a26a8ac0>
		.Grayscale -> <syft.ast.klass.Class object at 0x7f85a26a8b80>
		.Pad -> <syft.ast.klass.Class object at 0x7f85a26a8be0>
		.RandomAffine -> <syft.ast.klass.Class object at 0x7f85a26a8ca0>
		.RandomApply -> <syft.ast.klass.Class object at 0x7f85a26a8d60>
		.RandomCrop -> <syft.ast.klass.Class object at 0x7f85a26a8e20>
		.RandomGrayscale -> <syft.ast.klass.Class object at 0x7f85a26a8ee0>
		.RandomHorizontalFlip -> <syft.ast.klass.Class object at 0x7f85a26a8fa0>
		.RandomPerspective -> <syft.ast.klass

In [14]:
remote_torch

Module:
	.Tensor -> <syft.ast.klass.Class object at 0x7f85a297cf40>
	.BFloat16Tensor -> <syft.ast.klass.Class object at 0x7f85a297cfa0>
	.BoolTensor -> <syft.ast.klass.Class object at 0x7f85a2938040>
	.ByteTensor -> <syft.ast.klass.Class object at 0x7f85a29380a0>
	.CharTensor -> <syft.ast.klass.Class object at 0x7f85a2938100>
	.DoubleTensor -> <syft.ast.klass.Class object at 0x7f85a2938160>
	.FloatTensor -> <syft.ast.klass.Class object at 0x7f85a29381c0>
	.HalfTensor -> <syft.ast.klass.Class object at 0x7f85a2938220>
	.IntTensor -> <syft.ast.klass.Class object at 0x7f85a2938280>
	.LongTensor -> <syft.ast.klass.Class object at 0x7f85a29382e0>
	.ShortTensor -> <syft.ast.klass.Class object at 0x7f85a2938340>
	.nn -> Module:
		.Parameter -> <syft.ast.klass.Class object at 0x7f85a2938460>
		.Module -> <syft.ast.klass.Class object at 0x7f85a28cf820>
		.Conv2d -> <syft.ast.klass.Class object at 0x7f85a28cfd00>
		.Dropout2d -> <syft.ast.klass.Class object at 0x7f85a28d4160>
		.Linear -> <syft.

### The server request to see client's data

In [15]:
root_client_data_ptr = remote_torch.randn(size=(1, 100))
print(f'pointer to client data: {root_client_data_ptr}')
client_data = root_client_data_ptr.get()
print(client_data)
print(client_data.device)

pointer to client data: <syft.proxy.torch.TensorPointer object at 0x7f85a20daeb0>
tensor([[ 1.4839e+00, -1.4026e-01, -1.0380e+00, -8.6361e-01,  1.2877e+00,
          7.9215e-01, -5.7131e-01, -5.4436e-01, -8.0904e-01, -8.4752e-03,
         -4.3351e-01,  4.0938e-01, -1.4418e-01,  5.4918e-01,  1.2680e+00,
         -7.6164e-02, -2.4584e+00, -1.1221e+00, -7.2104e-01,  4.0913e-01,
          1.1193e+00,  2.4856e-01, -1.1446e+00, -3.7116e-01, -1.1473e+00,
          1.3001e+00, -5.2775e-01,  1.9871e-01,  7.1536e-01, -2.5062e-01,
         -1.1728e+00,  2.6915e-01, -9.2223e-03, -1.3356e-01, -8.9276e-01,
          2.9339e-01,  4.8440e-01,  2.9894e-03, -6.2730e-01, -2.2054e-01,
         -2.4610e-02,  4.7696e-01,  1.4238e+00, -6.6338e-01,  6.7872e-01,
          1.0820e+00,  2.5925e+00,  3.0458e+00,  2.0120e+00,  1.2231e+00,
         -4.1139e-01, -2.5692e+00,  1.9517e+00,  1.6609e+00,  5.5270e-03,
          2.1843e-01, -1.6698e+00, -9.1326e-01,  2.9642e-01, -1.8254e+00,
          5.3720e-01, -1.6095e

### Checking out client's device (cuda or CPU)

For root client

In [16]:
root_client_cuda_ptr = remote_torch.cuda.is_available()
root_client_cuda_ptr.id_at_location
bool_cuda = root_client_cuda_ptr.get_copy()
print(f'root client has CUDA? {bool_cuda}')

cuda_ptr = remote_torch.device("cuda" if bool_cuda else "cpu")
print(f'root client device: {cuda_ptr.type.get_copy()}')

root client has CUDA? True
root client device: cuda


What if the server requests a cuda tensor?

In [20]:
root_client_data_ptr = remote_torch.randn(size=(1, 100)).cuda()
print(f'pointer to client data: {root_client_data_ptr}')
try:for i, b in enumerate(tqdm(train_rdl_ptr)):
    if i<1:
        x, y = b[0], b[1]
        try:
            ic(x.get_copy(), y.get_copy())
        except Exception as e:
            ic(e)
        out = ecg_client_ptr(x)
        ic(out.cpu())
        # ic(X.to("cpu").get_copy(), y.to("cpu").get_copy())for i, b in enumerate(tqdm(train_rdl_ptr)):
    if i<1:
        x, y = b[0], b[1]
        try:
            ic(x.get_copy(), y.get_copy())
        except Exception as e:
            ic(e)
        out = ecg_client_ptr(x)
        ic(out.cpu())
        # ic(X.to("cpu").get_copy(), y.to("cpu").get_copy())for i, b in enumerate(test_rdl_ptr):
    if i<2:
        X, y = b[0], b[1]
        ic(X, y)
        ic(X.get_copy(), y.get_copy())asdfasdf
    client_data = root_client_data_ptr.get()
    print(client_data)
    print(client_data.device)
except TypeError as e:
    print(e)

pointer to client data: <syft.proxy.torch.TensorPointer object at 0x7f858d5f4940>
can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.


Check out device for normal client (somehow syft asks for request, which I don't know how to accept yet)

In [19]:
remote_torch2 = client.torch
bool_cuda_ptr2 = remote_torch2.cuda.is_available()
try:
    bool_cuda2 = bool_cuda_ptr2.get_copy()
    print(f'client has CUDA? {bool_cuda2}')
    cuda_ptr = remote_torch2.device("cuda" if bool_cuda else "cpu")
    print(f'client device: {cuda_ptr.type.get_copy()}')
except Exception as e:
    print(e)

[2021-09-16T08:25:46.561614+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: d9b3e1d5e4c84be3b06109281c5916f6>Please submit a request.
[2021-09-16T08:25:46.562207+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: d9b3e1d5e4c84be3b06109281c5916f6>Please submit a request.
[2021-09-16T08:25:46.563654+0000][CRITICAL][logger]][7646] You do not have permission to .get() Object with ID: <UID: d9b3e1d5e4c84be3b06109281c5916f6>Please submit a request.


You do not have permission to .get() Object with ID: <UID: d9b3e1d5e4c84be3b06109281c5916f6>Please submit a request.


### Sending data and do operations

In [ ]:
t1 = torch.tensor([1, 2])
t2 = torch.tensor([1, 3])
syft_list = List([t1, t2])
syft_list_ptr = syft_list.send(client)

In [ ]:
res = syft_list_ptr.get_copy()
res

In [ ]:
tensor_type='uint8'
op_name = '__add__'

In [ ]:
x = torch.tensor([-1, 0, 1, 2, 3, 4])
xp = x.send(client)
op_method = getattr(xp, op_name, None)
target_op_method = getattr(x, op_name)

print(op_method)
print(target_op_method)

In [ ]:
result = op_method(xp)  # op(xp, xp)
target_result = target_op_method(x)

local_result = result.get()

assert (local_result == target_result).all()

In [ ]:
local_result